# Power Outages

**Name(s)**: Karsin Dass & Cole Doyle

**Website Link**: https://keemarice.github.io/PowerOutages/

In [2]:
import pandas as pd
import numpy as np

import plotly.express as px
pd.options.plotting.backend = 'plotly'

# from lec_utils import * # Feel free to uncomment and use this. It'll make your plotly graphs look like ours in lecture!

## Introduction

In [19]:
outageFull = pd.read_csv('outage.csv', usecols = list(range(2, 56)), header = 0, skiprows=5,) #with units
print(outageFull.head())
print(outageFull.columns)
#print nmber of columns
print(outageFull.columns.size)

     YEAR  MONTH U.S._STATE POSTAL.CODE NERC.REGION      CLIMATE.REGION  \
0     NaN    NaN        NaN         NaN         NaN                 NaN   
1  2011.0    7.0  Minnesota          MN         MRO  East North Central   
2  2014.0    5.0  Minnesota          MN         MRO  East North Central   
3  2010.0   10.0  Minnesota          MN         MRO  East North Central   
4  2012.0    6.0  Minnesota          MN         MRO  East North Central   

  ANOMALY.LEVEL CLIMATE.CATEGORY                 OUTAGE.START.DATE  \
0       numeric              NaN  Day of the week, Month Day, Year   
1          -0.3           normal              Friday, July 1, 2011   
2          -0.1           normal              Sunday, May 11, 2014   
3          -1.5             cold         Tuesday, October 26, 2010   
4          -0.1           normal            Tuesday, June 19, 2012   

              OUTAGE.START.TIME  ... POPULATION POPPCT_URBAN POPPCT_UC  \
0  Hour:Minute:Second (AM / PM)  ...        NaN       

## Data Cleaning and Exploratory Data Analysis

### Cleaning

In [4]:
outageClean = outageFull[["YEAR", "U.S._STATE", "POSTAL.CODE", "NERC.REGION", "CAUSE.CATEGORY", "OUTAGE.DURATION", "DEMAND.LOSS.MW", "CUSTOMERS.AFFECTED"	]]
outageClean = outageClean.iloc[1:]
outageClean[['YEAR', 'OUTAGE.DURATION']].dropna()
outageClean['YEAR'] = pd.to_numeric(outageClean['YEAR'])
outageClean['OUTAGE.DURATION'] = pd.to_numeric(outageClean['OUTAGE.DURATION'])
outageClean['DEMAND.LOSS.MW'] = pd.to_numeric(outageClean['DEMAND.LOSS.MW'])
outageClean['CUSTOMERS.AFFECTED'] = pd.to_numeric(outageClean['CUSTOMERS.AFFECTED'])

outageClean

,YEAR,U.S._STATE,POSTAL.CODE,NERC.REGION,CAUSE.CATEGORY,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED
1,2011.0,Minnesota,MN,MRO,severe weather,3060.0,NaN,70000.0
2,2014.0,Minnesota,MN,MRO,intentional attack,1.0,NaN,NaN
3,2010.0,Minnesota,MN,MRO,severe weather,3000.0,NaN,70000.0
4,2012.0,Minnesota,MN,MRO,severe weather,2550.0,NaN,68200.0
5,2015.0,Minnesota,MN,MRO,severe weather,1740.0,250.0,250000.0
...,...,...,...,...,...,...,...,...
1530,2011.0,North Dakota,ND,MRO,public appeal,720.0,155.0,34500.0
1531,2006.0,North Dakota,ND,MRO,fuel supply emergency,NaN,1650.0,NaN
1532,2009.0,South Dakota,SD,RFC,islanding,59.0,84.0,NaN
1533,2009.0,South Dakota,SD,MRO,islanding,181.0,373.0,NaN


### Univariate Analysis

In [5]:
fig = px.histogram(outageClean, x = 'YEAR', nbins = 17)
fig.update_layout(title = "Power Outages per Year (2000 - 2016)", xaxis_title = "Year", yaxis_title = "Number of Outages")
fig.update_layout(
   xaxis = dict(
      tickmode = 'linear',
      tick0 = 2000,
      dtick = 1
   )
)
fig.show()



In [6]:
small = outageClean[outageClean["OUTAGE.DURATION"] < 10000]

fig = px.histogram(x = small['OUTAGE.DURATION'])
fig.update_layout(title = "Distribution of Outage Duration", xaxis_title = "Outage Duration (minutes)", yaxis_title = "Number of Outages")
fig.show()


### Bivariate Analysis

In [22]:
fig = px.scatter(outageClean, x='OUTAGE.DURATION', y='CUSTOMERS.AFFECTED', title='Outage Duration vs Customers Affected')
fig.update_layout(xaxis_title='Outage Duration (minutes)', yaxis_title='Customers Affected')
fig.show()

fig = px.scatter(outageClean, x='DEMAND.LOSS.MW', y='CUSTOMERS.AFFECTED', title='Demand Loss vs Customers Affected')
fig.update_layout(xaxis_title='Demand Loss (MW)', yaxis_title='Customers Affected')
fig.show()


In [7]:
durationstate = outageClean.groupby("POSTAL.CODE")["OUTAGE.DURATION"].mean().reset_index()

fig = px.choropleth(
    durationstate,
    locations="POSTAL.CODE",  
    locationmode="USA-states",  
    color="OUTAGE.DURATION",  # depper red = more customers affected
    scope="usa", 
    title="Average Outage Duration by State",
    color_continuous_scale="Reds",
    labels={"OUTAGE.DURATION": "Average Outage Duration (minutes)"},
)
fig.show()






In [8]:
durationregion = outageClean.groupby("NERC.REGION")["OUTAGE.DURATION"].mean().reset_index()

fig = px.bar(durationregion, x = 'NERC.REGION', y = 'OUTAGE.DURATION')
fig.update_layout(title = "Average Outage Duration by North American Electric Reliability Corporation (NERC) Regions", xaxis_title = "NERC Regions", yaxis_title = "Average Outage Duration (minutes)")

fig.show()

### Interesting Aggreates

In [9]:
outageClean.groupby('YEAR')['OUTAGE.DURATION'].mean().reset_index()

,YEAR,OUTAGE.DURATION
0,2000.0,2843.076923
1,2001.0,1272.071429
2,2002.0,4751.000000
3,2003.0,4652.434783
4,2004.0,4368.788732
5,2005.0,5288.944444
6,2006.0,3329.530303
7,2007.0,2336.666667
8,2008.0,4184.018182
9,2009.0,3660.519481


In [11]:
#group by day of the week
outageClean['DATE'] = pd.to_datetime(outageClean['YEAR'], format='%Y')
outageClean['DAY'] = outageClean['DATE'].dt.day_name()
outageClean['DAY'] = pd.Categorical(outageClean['DAY'], categories=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday'], ordered=True)
print(outageClean.groupby('DAY')['OUTAGE.DURATION'].mean().reset_index())


         DAY  OUTAGE.DURATION
0     Monday      2117.485294
1    Tuesday      2662.568841
2  Wednesday      3581.180000
3   Thursday      2721.417323
4     Friday      2718.816993
5   Saturday      2402.366071
6     Sunday      2278.824268


/var/folders/p_/yyb_zpwd6fj2v05f12wq70pc0000gn/T/ipykernel_75488/3888886289.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



### Baseline Model

In [34]:
numeric_columns = outageClean.select_dtypes(include=['number'])

# Compute the correlation matrix
correlation_matrix = numeric_columns.corr()

# Extract correlations with 'OUTAGE.DURATION', sort them, and drop the target itself
correlated_features = correlation_matrix['OUTAGE.DURATION'].drop('OUTAGE.DURATION').sort_values(ascending=False)

# Display the features most correlated with 'OUTAGE.DURATION'
print("Features most positively correlated with OUTAGE.DURATION:")
print(correlated_features[correlated_features > 0].head())

print("\nFeatures most negatively correlated with OUTAGE.DURATION:")
print(correlated_features[correlated_features < 0].head())



Features most positively correlated with OUTAGE.DURATION:
CUSTOMERS.AFFECTED    0.261916
DEMAND.LOSS.MW        0.026798
Name: OUTAGE.DURATION, dtype: float64

Features most negatively correlated with OUTAGE.DURATION:
YEAR   -0.144047
Name: OUTAGE.DURATION, dtype: float64


In [36]:
#linear regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = outageClean[['CUSTOMERS.AFFECTED']]
X = X.fillna(X.mean())
y = outageClean['OUTAGE.DURATION']
y = y.fillna(y.mean())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(mean_squared_error(y_test, y_pred))

fig = px.scatter(x = X_test['CUSTOMERS.AFFECTED'], y = y_test)
fig.add_scatter(x = X_test['CUSTOMERS.AFFECTED'], y = y_pred, mode = 'lines')
fig.update_layout(title = "Customers Affected vs Outage Duration", xaxis_title = "Customers Affected", yaxis_title = "Outage Duration (minutes)")
fig.show()

69737117.95445843


In [58]:
#lets do multiple linear regression, with every column except for the target column
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = outageClean[['CUSTOMERS.AFFECTED', 'DEMAND.LOSS.MW', 'YEAR']]  
X = X.fillna(X.mean())  
y = outageClean['OUTAGE.DURATION']
y = y.fillna(y.mean()) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared = False)
print(f"Root Mean Squared Error: {rmse}")


print(model.coef_)
print(model.intercept_)
#print formula
print(f"y = {model.intercept_} + {model.coef_[0]} * x1 + {model.coef_[1]} * x2 + {model.coef_[2]} * x3")


Root Mean Squared Error: 8356.46177555498
[ 2.32836191e-03 -9.77451988e-02 -1.90873166e+02]
385898.327156706
y = 385898.327156706 + 0.0023283619057560337 * x1 + -0.09774519878674237 * x2 + -190.8731660052654 * x3


/Users/karsi/miniforge3/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.



In [70]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import pandas as pd

X = outageClean[['CAUSE.CATEGORY', 'NERC.REGION', 'U.S._STATE', 'YEAR']]
y = outageClean['OUTAGE.DURATION']  # Target variable

# Handle missing values
X['YEAR'] = X['YEAR'].fillna(0).astype(int) 
X['CAUSE.CATEGORY'] = X['CAUSE.CATEGORY'].fillna("Missing")
X['NERC.REGION'] = X['NERC.REGION'].fillna("Missing")
X['U.S._STATE'] = X['U.S._STATE'].fillna("Missing")
y = y.fillna(y.mean())  

label_encoder_state = LabelEncoder()
label_encoder_cause = LabelEncoder()
label_encoder_nerc = LabelEncoder()

X['U.S._STATE'] = label_encoder_state.fit_transform(X['U.S._STATE'])
X['CAUSE.CATEGORY'] = label_encoder_cause.fit_transform(X['CAUSE.CATEGORY'])
X['NERC.REGION'] = label_encoder_nerc.fit_transform(X['NERC.REGION'])

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['YEAR', 'U.S._STATE', 'CAUSE.CATEGORY', 'NERC.REGION']),  # Scale numeric features
    ]
)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lasso', Lasso(alpha=0.1))
])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
#rmse not mse
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Root Mean Squared Error (RMSE): {rmse}")
lasso_model = pipeline.named_steps['lasso']
encoded_feature_names = ['YEAR', 'U.S._STATE', 'CAUSE.CATEGORY', 'NERC.REGION']
coefficients = pd.DataFrame({
    'Feature': encoded_feature_names,
    'Coefficient': lasso_model.coef_
}).sort_values(by='Coefficient', key=abs, ascending=False)

# print bonanza
print("Most influential features:")
print(coefficients.head(10))
print(f"Number of features: {len(encoded_feature_names)}")
print(f"Number of non-zero coefficients: {len(coefficients[coefficients['Coefficient'] != 0])}")


Root Mean Squared Error (RMSE): 5008.322609570352
Most influential features:
          Feature  Coefficient
3     NERC.REGION  -723.365315
0            YEAR  -517.029377
1      U.S._STATE   297.986658
2  CAUSE.CATEGORY   165.481130
Number of features: 4
Number of non-zero coefficients: 4


/var/folders/p_/yyb_zpwd6fj2v05f12wq70pc0000gn/T/ipykernel_75488/1871802544.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/p_/yyb_zpwd6fj2v05f12wq70pc0000gn/T/ipykernel_75488/1871802544.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/p_/yyb_zpwd6fj2v05f12wq70pc0000gn/T/ipykernel_75488/1871802544.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th